In [1]:

import numpy as np
from abc import abstractmethod

class Classifier:

    @abstractmethod
    def fit(self,X,y):
        pass

    @abstractmethod
    def predict(self,X):
        pass
import numpy as np
from abc import abstractmethod

from sklearn.datasets import load_iris

class Classifier:

    @abstractmethod
    def fit(self,X,y):
        pass

    @abstractmethod
    def predict(self,X):
        pass
class ClassifEuclid(Classifier):
    
    def __init__(self,labels=[]):
        """Constructor de la clase
        labels: lista de etiquetas de esta clase (argumento necesario)"""
        self.labels = labels
        self.Z = None # Array de centroides

    def fit(self, X, y):
        """Entrena el clasificador
        X: matriz numpy cada fila es un dato, cada columna una medida
        y: vector de etiquetas, tantos elementos como filas en X
        retorna objeto clasificador"""
        n = np.zeros(len(self.labels)) # Contador de ocurrencias de cada clase
        self.Z = np.zeros((len(self.labels), X.shape[1]))
        # Calcular la media: 
        # Sumar las ocurrencias de cada clase en self.Z
        for yi, Xi in zip(y, X):
            n[yi] = n[yi] + 1
            self.Z[yi] = self.Z[yi] + Xi
        # Dividir cada sumatorio entre el númeo de ocurrencias
        self.Z = self.Z / n[:,np.newaxis]
        return self

    def predict(self, X):
        """Estima el grado de pertenencia de cada dato a todas las clases 
        X: matriz numpy cada fila es un dato, cada columna una medida del vector de caracteristicas. 
        Retorna una matriz, con tantas filas como datos y tantas columnas como clases tenga
        el problema, cada fila almacena los valores pertenencia de un dato a cada clase"""
        # Calcular la distancia de cada fila a cada centroide
        aux = X[:,None]-self.Z
        return np.sqrt(np.einsum('abc,abc->ab', aux, aux))
        #return np.sqrt(np.sum(np.power(X[:,np.newaxis] - self.Z, 2), axis=2))

    def pred_label(self, X):
        """Estima la etiqueta de cada dato. La etiqueta puede ser un entero o bien un string.
        X: matriz numpy cada fila es un dato, cada columna una medida
        retorna un vector con las etiquetas de cada dato"""
        # Devuelve un array con el índice con valor mínimo de cada fila.
        # Cada índice se corresponde con la clase a la que pertenece.
        return np.argmin(self.predict(X), axis=1)

    def num_aciertos(self, X, y):
        """Cuenta el numero de aciertos del clasificador para un conjunto de datos X.
        X: matriz de datos a clasificar
        y: vector de etiquetas correctas"""
        # Contar el número de datos iguales en ambos vectores
        return np.sum(self.pred_label(X)==y)

class ClassifEstadistico(Classifier):
    
    def __init__(self,labels=[]):
        """Constructor de la clase
        labels: lista de etiquetas de esta clase (argumento necesario)"""
        self.labels = labels
        self.cov = None # Array de medias
        self.mu = None # Array de matrices de covarianza de cada clase
        self.cov_inv = None # Array de matrices de covarianza inversas
        self.det = None # Array de determinantes de las matrices de covarianza
        self.a = None
        self.b = None
        self.c = None

    def fit(self,X,y):
        """Entrena el clasificador
        X: matriz numpy cada fila es un dato, cada columna una medida
        y: vector de etiquetas, tantos elementos como filas en X
        retorna objeto clasificador"""
        n_labels = len(self.labels)
        n_caracteristicas = X.shape[1]
        self.mu = np.empty((n_labels, n_caracteristicas))
        self.cov = np.empty((n_labels, n_caracteristicas, n_caracteristicas))
        self.cov_inv = np.empty((n_labels, n_caracteristicas, n_caracteristicas))
        self.det = np.empty(n_labels)
        self.a = np.empty((n_labels, n_caracteristicas, n_caracteristicas))
        self.b = np.empty((n_labels, n_caracteristicas))
        self.c = np.empty(n_labels)
        for c in range(len(self.labels)):
            self.cov[c] = np.cov(X[y==c], rowvar=False)
            self.mu[c] = np.mean(X[y==c], axis=0)
            self.cov_inv[c] = np.linalg.inv(self.cov[c])
            self.det[c] = np.linalg.det(self.cov[c])
            self.a[c] = -.5 * self.cov_inv[c]
            self.b[c] = self.mu[c].T @ self.cov_inv[c]
            self.c[c] = -.5 * (self.mu[c].T @ self.cov_inv[c] @ self.mu[c]) -.5 * np.log(self.det[c]) + np.log(np.sum(y==c)/X.shape[0])
        return self

    def predict(self,X):
        """Estima el grado de pertenencia de cada dato a todas las clases 
        X: matriz numpy cada fila es un dato, cada columna una medida del vector de caracteristicas. 
        Retorna una matriz, con tantas filas como datos y tantas columnas como clases tenga
        el problema, cada fila almacena los valores pertenencia de un dato a cada clase"""
        #return (np.einsum('ab,cdb->acd', X, self.a)@X.T + self.b@X.T + self.c[:,np.newaxis]).T
        #return (np.einsum('abc,ac->ab', np.einsum('ab,cdb->acd', X, self.a), X) + (self.b@X.T).T + self.c[np.newaxis,:]).T
        #return np.einsum('ab,cdb,ad->ac', X, self.a, X) + (self.b@X.T).T + self.c[None,:]
        return np.einsum('ab,cdb,ad->ac', X, self.a, X) + np.einsum('ab,cb->ca', self.b, X) + self.c[None,:]
        #return (np.diagonal(X@self.a@X.T, axis1=1, axis2=2) + self.b@X.T + self.c[:,np.newaxis]).T

    def pred_label(self,X):
        """Estima la etiqueta de cada dato. La etiqueta puede ser un entero o bien un string.
        X: matriz numpy cada fila es un dato, cada columna una medida
        retorna un vector con las etiquetas de cada dato"""
        # Devuelve un array con el índice con valor mínimo de cada fila.
        # Cada índice se corresponde con la clase a la que pertenece.
        return np.argmax(self.predict(X), axis=1)

    def num_aciertos(self,X,y):
        """Cuenta el numero de aciertos del clasificador para un conjunto de datos X.
        X: matriz de datos a clasificar
        y: vector de etiquetas correctas"""
        # Contar el número de datos iguales en ambos vectores
        return np.sum(self.pred_label(X)==y)


In [2]:
"""PRUEBAS"""
"""
mu = np.empty((3, 4))
cov = np.empty((3, 4, 4))
covi = np.empty((3, 4, 4))
det = np.empty(3)
a = np.empty((3, 4, 4))
b = np.empty((3, 4))
c = np.empty(3)
for ci in range(3):
    mu[ci] = np.mean(X[y==ci], axis=0)
    cov[ci] = np.cov(X[y==ci], rowvar=False)
    covi[ci] = np.linalg.inv(cov[ci])
    det[ci] = np.linalg.det(cov[ci])
    a[ci] = -.5 * covi[ci]
    b[ci] = mu[ci].T @ covi[ci]
    c[ci] = -.5 * (mu[ci] @ covi[ci] @ mu[ci].T) -.5 * np.log(det[ci]) + np.log(np.sum(y==ci)/X.shape[0])

"""

'\nmu = np.empty((3, 4))\ncov = np.empty((3, 4, 4))\ncovi = np.empty((3, 4, 4))\ndet = np.empty(3)\na = np.empty((3, 4, 4))\nb = np.empty((3, 4))\nc = np.empty(3)\nfor ci in range(3):\n    mu[ci] = np.mean(X[y==ci], axis=0)\n    cov[ci] = np.cov(X[y==ci], rowvar=False)\n    covi[ci] = np.linalg.inv(cov[ci])\n    det[ci] = np.linalg.det(cov[ci])\n    a[ci] = -.5 * covi[ci]\n    b[ci] = mu[ci].T @ covi[ci]\n    c[ci] = -.5 * (mu[ci] @ covi[ci] @ mu[ci].T) -.5 * np.log(det[ci]) + np.log(np.sum(y==ci)/X.shape[0])\n\n'

## Código para cargar la base de datos desde un fichero en drive:

In [3]:
# from google.colab import drive
# from sklearn.datasets import load_iris

# drive.mount('/content/gdrive')
# drive_dir = '/content/gdrive/My Drive/Colab Notebooks/'

# import pandas as pd
# # Cargar las características de la BD
# X = np.genfromtxt(drive_dir + 'irisData.txt', delimiter='\t')[:, :-1]
# print("X: \n" + str(X))

# # Leer las etiquetas y convertirlas a enteros
# y = np.genfromtxt(drive_dir + 'irisData.txt', dtype=str, delimiter='\t')[:, -1]
# y = pd.factorize(y)[0]
# print("y: \n" + str(y))

## Código para cargar la base de datos Iris directamente desde sklearn:

In [4]:
from sklearn.datasets import load_iris
dataset = load_iris()
X = dataset.data
#print("X: \n" + str(X))
y = dataset.target
#print("y: \n" + str(y))

Entrenamiento, predicción y evaluación

In [5]:
from sklearn.datasets import load_iris
from sklearn.datasets import load_wine
from sklearn.datasets import load_breast_cancer
from sklearn.neighbors import NearestCentroid
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

"""
Clasificador distancia euclídea
"""

print("", "#"*76, "\tClasificador distancia euclídea", "#"*76, sep="\n")
dataset = load_iris()
X = dataset.data
y = dataset.target

# 1. Cargar los datos de la base de datos de entrenamiento
a = ClassifEuclid(dataset.target_names)

# 2. Entrenar el clasificador
train = a.fit(np.array(X), y)
#print("-"*32, "centroides", "-"*32, "\n", train.Z, "\n")

# 3. Predecir empleando la base de datos de entrenamiento (X)
pred = a.predict(X)
#print("-"*32, "distancias", "-"*32, "\n", pred, "\n")
result = a.pred_label(X)
#print("-"*32, "etiquetas", "-"*33, "\n", result, "\n")

# 4. Evaluar el clasificador calculando el porcentaje de datos correctamente clasificados
n_aciertos = a.num_aciertos(X, y)
print("-"*76, "\niris:   Aciertos: ", n_aciertos, "/", len(y), " (", "%.2f" % ((n_aciertos / len(y))*100), "%)", sep='')

# Comparación con el clasificador de sklearn
nc = NearestCentroid()
nc.fit(X, y)
print(" "*10, "Clasificador sklearn: ", np.sum(nc.predict(X)==y), " aciertos", sep='')


# wine
dataset = load_wine()
X = dataset.data
y = dataset.target
b = ClassifEuclid(dataset.target_names)
train = b.fit(np.array(X), y)
#print("-"*32, "centroides", "-"*32, "\n", train.Z, "\n")
pred = b.predict(X)
#print("-"*32, "distancias", "-"*32, "\n", pred, "\n")
result = b.pred_label(X)
#print("-"*32, "etiquetas", "-"*33, "\n", result, "\n")
n_aciertos = b.num_aciertos(X, y)
print("-"*76, "\nwine:   Aciertos: ", n_aciertos, "/", len(y), " (", "%.2f" % ((n_aciertos / len(y))*100), "%)", sep='')
nc = NearestCentroid()
nc.fit(X, y)
print(" "*10, "Clasificador sklearn: ", np.sum(nc.predict(X)==y), " aciertos", sep='')


# cancer
dataset = load_breast_cancer()
X = dataset.data
y = dataset.target
c = ClassifEuclid(dataset.target_names)
train = c.fit(np.array(X), y)
#print("-"*32, "centroides", "-"*32, "\n", train.Z, "\n")
pred = c.predict(X)
#print("-"*32, "distancias", "-"*32, "\n", pred, "\n")
result = c.pred_label(X)
#print("-"*32, "etiquetas", "-"*33, "\n", result, "\n")
n_aciertos = c.num_aciertos(X, y)
print("-"*76, "\ncancer:   Aciertos: ", n_aciertos, "/", len(y), " (", "%.2f" % ((n_aciertos / len(y))*100), "%)", sep='')
nc = NearestCentroid()
nc.fit(X, y)
print(" "*10, "Clasificador sklearn: ", np.sum(nc.predict(X)==y), " aciertos", sep='')


"""
Clasificador estadístico
"""
print("", "#"*76, "\tClasificador estadístico", "#"*76, sep="\n")
# iris
dataset = load_iris()
X = dataset.data
y = dataset.target
b = ClassifEstadistico(dataset.target_names)
train = b.fit(np.array(X), y)
pred = b.predict(X)
result = b.pred_label(X)
n_aciertos = b.num_aciertos(X, y)
print("-"*76, "\niris:   Aciertos: ", n_aciertos, "/", len(y), " (", "%.2f" % ((n_aciertos / len(y))*100), "%)", sep='')
c_est = QuadraticDiscriminantAnalysis()
c_est.fit(X, y)
print(" "*10, "Clasificador sklearn: ", np.sum(c_est.predict(X)==y), " aciertos", sep='')

# wine
dataset = load_wine()
X = dataset.data
y = dataset.target
b = ClassifEstadistico(dataset.target_names)
train = b.fit(np.array(X), y)
pred = b.predict(X)
result = b.pred_label(X)
n_aciertos = b.num_aciertos(X, y)
print("-"*76, "\nwine:   Aciertos: ", n_aciertos, "/", len(y), " (", "%.2f" % ((n_aciertos / len(y))*100), "%)", sep='')
c_est = QuadraticDiscriminantAnalysis()
c_est.fit(X, y)
print(" "*10, "Clasificador sklearn: ", np.sum(c_est.predict(X)==y), " aciertos", sep='')

# cancer
dataset = load_breast_cancer()
X = dataset.data
y = dataset.target
b = ClassifEstadistico(dataset.target_names)
train = b.fit(np.array(X), y)
pred = b.predict(X)
result = b.pred_label(X)
n_aciertos = b.num_aciertos(X, y)
print("-"*76, "\ncancer:   Aciertos: ", n_aciertos, "/", len(y), " (", "%.2f" % ((n_aciertos / len(y))*100), "%)", sep='')
c_est = QuadraticDiscriminantAnalysis()
c_est.fit(X, y)
print(" "*10, "Clasificador sklearn: ", np.sum(c_est.predict(X)==y), " aciertos", sep='')




############################################################################
	Clasificador distancia euclídea
############################################################################
----------------------------------------------------------------------------
iris:   Aciertos: 139/150 (92.67%)
          Clasificador sklearn: 139 aciertos
----------------------------------------------------------------------------
wine:   Aciertos: 129/178 (72.47%)
          Clasificador sklearn: 129 aciertos
----------------------------------------------------------------------------
cancer:   Aciertos: 507/569 (89.10%)
          Clasificador sklearn: 507 aciertos

############################################################################
	Clasificador estadístico
############################################################################
----------------------------------------------------------------------------
iris:   Aciertos: 147/150 (98.00%)
          Clasificador sklearn: 147 aciertos
-

Resultados de los tres experimentos (clasificador distancia euclídea):

| Base de datos | Número de aciertos | Porcentaje de aciertos |
| --- | --- | --- |
| Iris   | 139| 92.67|
| Wine   | 129| 72.47|
| Cancer | 507| 89.10|

Resultados de los tres experimentos (clasificador estadístico):

| Base de datos | Número de aciertos | Porcentaje de aciertos |
| --- | --- | --- |
| Iris   | 147| 98.00|
| Wine   | 177| 99.44|
| Cancer | 554| 97.37|
